In [ ]:
import numpy as np
import pandas as pd

resList = np.load('Resident_gov3_res12.npy', allow_pickle=True)

In [ ]:
res_percent = []
res_self_relocation_year = []
res_opt_relocation_year = []
for res in resList:
    if res.optimotiFlag == True:
        optimal_relocation_year = res.optmotiMoveYear

        res_opt_relocation_year.append(res.optmotiMoveYear)
        res_self_relocation_year.append(res.selfMoveYear)

        res_percent.append(res.Subsidyneeded[optimal_relocation_year] / res.replacementcost)

import matplotlib.pyplot as plt
plt.scatter(res_self_relocation_year, res_percent)
plt.show()

In [ ]:
# plot the percent for those choosing never to relocate
never_self_relocate = [1 if x> 100 else 0 for x in res_self_relocation_year ]
never_self_relocation_percent = [never_self_relocate[i] * res_percent[i] for i in range(len(res_percent))]
neverself_optrelocate = [res_opt_relocation_year[i] if never_self_relocate[i] == 1 else 200 for i in range(len(res_percent))]

y_never = [i for i in never_self_relocation_percent if i != 0]
x_never = [i for i in neverself_optrelocate if i != 200]

plt.scatter(x_never, y_never)
plt.xticks(np.arange(0, 21, 1))
plt.title('Relocation Percent VS Optimally Motivated Residents - When residents never self-relocate')
plt.xlabel('Optimal Relocation Year')
plt.ylabel('Subsidy Percent')
plt.show()

In [ ]:
# creat dataframe
df = {}
year_list = list(np.arange(0, 20, 1))
for year in year_list:
    df[year] = []
for i in range(len(x_never)):
    df[x_never[i]].append(y_never[i])

labels, values = [], []
for key, value_list in df.items():
    labels.extend([key] * len(value_list))
    values.extend(value_list)

# Plot
import seaborn as sns
plt.figure(figsize=(10, 6))
sns.violinplot(x=labels, y=values)
plt.xlabel('Optimal Relocation Year')
plt.ylabel('Optimal subsidy/Replacement cost')
plt.title("For residents never self-relocating")
plt.ylim(0, 1)
plt.show()

In [ ]:
# plot the percent for those whose self-relocation year is earlier than year 20
self_relocate = [1 if x< 21 else 0 for x in res_self_relocation_year ]
self_relocation_percent = [self_relocate[i] * res_percent[i] for i in range(len(res_percent))]
self_optrelocate = [res_opt_relocation_year[i] if self_relocate[i] == 1 else 200 for i in range(len(res_percent))]

self_relocation_year = [res_self_relocation_year[i] if self_relocate[i] == 1 else 200 for i in range(len(res_percent))]

x_will = [i for i in self_optrelocate if i != 200]
x_will2 = [i for i in self_relocation_year if i != 200]
y_will = [i for i in self_relocation_percent if i != 0]

plt.scatter(x_will, y_will)
plt.xticks(np.arange(0, 21, 1))
plt.title('Relocation Percent VS Optimally Motivated Residents - When residents will self-relocate')
plt.xlabel('Optimal Relocation Year')
plt.ylabel('Subsidy Percent')
plt.show()

In [ ]:
# the scatter plot is not intuitive, design densitity curves to illustrate distribution
# creat dataframe
df = {}
year_list = list(np.arange(0, 20, 1))
for year in year_list:
    df[year] = []
for i in range(len(x_will)):
    df[x_will[i]].append(y_will[i])

labels, values = [], []
for key, value_list in df.items():
    labels.extend([key] * len(value_list))
    values.extend(value_list)

# Plot
import seaborn as sns
plt.figure(figsize=(10, 6))
sns.violinplot(x=labels, y=values)
plt.xlabel('Optimal Relocation Year')
plt.ylabel('Optimal subsidy/Replacement cost')
plt.title("For residents whose self-relocation are before year 20")
plt.ylim(0, 1)
plt.show()

In [ ]:
df = {}
year_list = list(np.arange(0, 21, 1))
for year in year_list:
    df[year] = []
for i in range(len(x_will2)):
    df[x_will2[i]].append(y_will[i])

labels, values = [], []
for key, value_list in df.items():
    labels.extend([key] * len(value_list))
    values.extend(value_list)

# Plot
import seaborn as sns
plt.figure(figsize=(10, 6))
sns.violinplot(x=labels, y=values)
plt.xlabel('Self Relocation Year')
plt.ylabel('Optimal subsidy/Replacement cost')
plt.title("For residents whose self-relocation are before year 20")
plt.ylim(0, 1)
plt.show()

In [ ]:
# this part is going to be displaying overall bcr and subsidy, for the optimal subsidy plan
# Firstly, create a dictionary, storing for residents motivated to relocate under the optimal subsidy plan. Elements being stored include: id, EAD reduction, subsidy, bcr; after the dictionary is generated, sort the data by bcr
import pandas as pd
df_fix = pd.read_csv('Fixed_Relocated_Residents_8_gov3_res12.csv')
df_fix_motivated = df_fix[df_fix['Fix_Motivated_Or_Not'] == 1]
print(df_fix_motivated.shape)
# calculate the bcr and sort
df_fix_motivated['BCR'] = df_fix_motivated['Fix_EAD_Reduction']/df_fix_motivated['Fix_Subsidy_Discounted']
# sort the dataframe by ranking the 'BCR'
df_fix_motivated_sorted = df_fix_motivated.sort_values(by = 'BCR', ascending=False)
# obtain two new different columns named 'Accumulated BCR' and 'Accumulated Subsidy'
df_fix_motivated_sorted['Subsidy_cumsum'] = df_fix_motivated_sorted['Fix_Subsidy_Discounted'].cumsum()
df_fix_motivated_sorted['EAD_cumsum'] = df_fix_motivated_sorted['Fix_EAD_Reduction'].cumsum()
df_fix_motivated_sorted['BCR_Accumulated'] = df_fix_motivated_sorted['EAD_cumsum']/df_fix_motivated_sorted['Subsidy_cumsum']
# visualize the figure
import matplotlib.pyplot as plt
plt.plot(df_fix_motivated_sorted['Subsidy_cumsum'], df_fix_motivated_sorted['BCR_Accumulated'])
plt.xlabel('Accumulated Subsidy')
plt.ylabel('Accumulated Benefit-Cost Ratio')
plt.title('Landscape 8, Fixed subsidy plan')
plt.show()

In [ ]:
df_opt = pd.read_csv('Optimal_Relocated_Residents_8_gov3_res12.csv')
df_opt_motivated = df_opt[df_opt['Opt_Motivated_Or_Not'] == 1]
print(df_opt_motivated.shape)
# drop the item where the subsidy is 0
df_opt_motivated = df_opt_motivated[df_opt_motivated['Opt_Subsidy_Discounted'] != 0]
# calculate the bcr and sort
df_opt_motivated['BCR'] = df_opt_motivated['Opt_EAD_Reduction']/df_opt_motivated['Opt_Subsidy_Discounted']
# sort the dataframe by ranking the 'BCR'
df_opt_motivated_sorted = df_opt_motivated.sort_values(by = 'BCR', ascending = False)
# obtain two new different columns named 'Accumulated BCR' and 'Accumulated Subsidy'
df_opt_motivated_sorted['Subsidy_cumsum'] = df_opt_motivated_sorted['Opt_Subsidy_Discounted'].cumsum()
df_opt_motivated_sorted['EAD_cumsum'] = df_opt_motivated_sorted['Opt_EAD_Reduction'].cumsum()
df_opt_motivated_sorted['BCR_Accumulated'] = df_opt_motivated_sorted['EAD_cumsum']/df_opt_motivated_sorted['Subsidy_cumsum']
# visualize the figure
import matplotlib.pyplot as plt
plt.plot(df_opt_motivated_sorted['Subsidy_cumsum'], df_opt_motivated_sorted['BCR_Accumulated'])
plt.xlabel('Accumulated Subsidy')
plt.ylabel('Accumulated Benefit-Cost Ratio')
plt.title('Landscape 8, Optimal subsidy plan')
plt.show()

In [ ]:
plt.figure()
plt.scatter(df_fix_motivated_sorted['Subsidy_cumsum'], df_fix_motivated_sorted['BCR_Accumulated'], label = 'Fixed', color = 'blue')
plt.scatter(df_opt_motivated_sorted['Subsidy_cumsum'], df_opt_motivated_sorted['BCR_Accumulated'], label = 'Opt', color = 'orange')
plt.legend()
plt.show()

In [ ]:
idlist = [352005, 704739, 97885, 646743, 685146]
result = []
## create a dictionary mappings ids to objects
id_to_object = {obj.idx: obj for obj in resList}
## lookup the objects by their ids
for id in idlist:
    id_result = {}
    id_result['id'] = id
    obj = id_to_object.get(id)
    if obj:
        id_result['rc1'] = obj.replacementcost
        id_result['rc2'] = obj.relocationcost
        id_result['eadlist'] = obj.ead
        id_result['self_year'] = obj.selfMoveYear
        id_result['fix_year'] = obj.motiMoveYear
        id_result['opt_year'] = obj.optmotiMoveYear
        id_result['opt_subsidy'] = obj.Subsidyneeded[obj.optmotiMoveYear]
    result.append(id_result)
result = pd.DataFrame(result)
print(result)

In [ ]:
# plot the year EAD for those structures
year = np.arange(0, 51, 1)
decision = np.arange(0, 21, 1)
for id in idlist:
    ead = result[result['id'] == id]['eadlist'].values[0]
    replacement = result[result['id'] == id]['rc1'].values[0]
    relocation = result[result['id'] == id]['rc2'].values[0]
    future_ead = [sum([ead[j]/1.12**(j-i) for j in np.arange(i, i+31)]) for i in decision]

    cost = [replacement+relocation for i in decision]
    # fig, ax = plt.subplots()
    plt.plot(year, ead, label = 'yearly undiscounted ead', color = 'red')

    # # get second axes
    # ax2 = ax.twinx()
    # plt.plot(decision, future_ead, label = 'future flood loss', color = 'darkorange')
    # plt.plot(decision, cost, label = 'RC1+RC2', color = 'red')

    plt.legend()
    plt.show()

In [ ]:
# plot the government loss curve for those structures
year = np.arange(0, 51, 1)
decision = np.arange(0, 21, 1)
for id in idlist:
    obj = id_to_object.get(id)
    if obj:
        print('Resident')
        # print(obj.Subsidyneeded)
        ead = obj.ead
        # print('ead', ead)
        # if there is no motivated relocation, what is the accumulated loss that government has to bear in every year
        past_loss =  [sum([ead[j]/1.05**j for j in np.arange(0, i)]) for i in decision]
        # print('past loss', past_loss)
        # government loss as a function of both flood losses and subsidy expenses
        loss_subsidy = [sum([ead[j]/1.03**j for j in np.arange(0, i)])+ obj.Subsidyneeded[i]/1.05**i for i in decision]
        # print('loss and subsidy', loss_subsidy)
    plt.figure()
    plt.plot(decision, past_loss, label = 'past loss from the government view', color = 'darkorange')
    plt.plot(decision, loss_subsidy, label = 'past loss and subsidy from the government view', color = 'red')
    plt.legend()
    plt.show()